# <u> Chapter 2: Aperture photometry</u>

<div class="alert alert-block alert-info">

### This Notebook will lead you through exercises to explore signal-to-noise in imaging data.
### In this notebook, you will:

-  learn how to perform aperture photomotry and sky subtraction
    
-  learn how to choose appropriate source aperture and sky aperture sizes

-  learn how to calculate errors for your photometry

-  look into the difference between source and sky noise

-  test your aperture photometry functions on simulated datasets

### If you have time and want to take go further with your analysis:

-  test your aperture photometry functions on difficult datasets, such as crowded fields and noisy datasets

### What's next:
after you have completed this notebook:

-  finalize your own aperture photometry function to use for Coursework B


### Table of contents:

- [Background: Photometry in Astronomy](#sec:bg)
    
- [1. Aperture photometry - step by step](#sec:stepbystep)

- [1.1 Defining the aperture](#sec:aperture)

- [1.2 Measuring fluxes](#sec:flux)
    
- [1.3 Estimating sky levels](#sec:sky)

- [2. Practice](#sec:practice)

    - [Advanced practice: noisy images](#sec:practice-noise)
    - [Advanced practice: crowded fields](#sec:practice-crowded)
- [Summary](#sec:summary)

[The aperture photometry function required for Coursework A](#aperturefunction)
</div>

In [ ]:
### Necessary imports fore this notebooks.
# ImageSimulator is a supplemtary set of code used to simulate the test images used 
import ImageSimulator
#Photutils is the model we will use for aperture photometry.
#The documentation is available here: https://photutils.readthedocs.io/en/stable/index.html
from photutils import aperture_photometry
import photutils

<a id="sec:bg"></a>
## Background - Photometry in Astronomy

For your research project, you will be working with photometry data. This refers to measuring the flux or magnitude of a source in an image. In your project, you will be working with point sources (that means sources that are unresolved at the resolution of the image).

#### Photometry in astronomy is used to:

$\bullet$ measure the flux of a source in a specific band, which can be used to determine the luminosity of a source

$\bullet$ contruct a so-called spectral energy distribution that measures the flux from a source as a function of wavelength or frequency. This can be used to determine intrinsic phyiscal properties of a source, such as its temperature, chemical composition, or in case of a galaxy, the stellar population.

$\bullet$ measure how an object's light evolves with time (often called a lightcurve), which can be used to study transient events such as supernovea, stochastic processes such as accretion onto black holes as well as dynamical systems such as binary stars and exoplanets. The latter example is what you will be working on in your research project. 

#### Signal in astronomical images:

Photometry is measured on images of the sky. Emission detected in such an image may come from:

$\bullet$ the source itself

$\bullet$ other sources in the field

$\bullet$ sky emission, generally uniform across the field

$\bullet$ scatter emission originating from within the instrument

$\bullet$ the bias: a signal added in charged coupled device (CCD) devices to avoid negative counts

#### The signal is affected by:

$\bullet$ differences in transmission through the atmosphere, i.e., only a fraction of flux will arrive on the ground. This is dealt with by measuring the flux relative to a star in the field with known magnitude/flux. You will work on this in the DataReduction section. In your research case, you will be using space-based photometry.

$\bullet$ differences in transmission through the instrument that vary across the field. This is corrected for using so-called flat-fields. Flat fields are images taken of the sky at dawn, these are used to correct for the differences in transmission. You will see an example of this un the DataReduction section. 

#### Spatial resolution in astronomical imaging:

Another important effect is that the image is effectively convoluted by the so-called point-spread function. This means that all point sources are detected as extended sources, here, we treat them as two-d gaussians:

$ PSF(x, y) = \frac{1}{\sigma^2 \sqrt{2\pi}} exp\left[ \frac{(x-x_0)^2 + (y-y_0)^2}{2\sigma^2}\right] $

where $x_0,y_0$ is the position and $\sigma$ the standard devition. In astronomy, rather than the standard deviation, we often use the so-called full-width at half maximum which is:

$ FWHM = 2 \sqrt{ln 2}\sigma \approx 2.35\sigma$.

#### A side note on units:

Astronomical images can have a range of different units, these can be counts/s. These are related through the gain, which turns counts or electrons per second into photons per second. However, when calculating errors, we are calculating Poisson noise, which is a counting noise error. This error needs to be calculated using the integrated number of photons in the exposure. In this notebook, we will be working with images in units of photons. 

#### Photometry and errors:

To perform aperture photometry, we therefore need to isolate the source emission from other signal and correct for losses in transmission (the latter will be covered in the Data Reduction section).

Additionally, we will need to calculate the error in our measurement. The main source of noise in astronomical data is Poisson or shot noise. The uncertainty $\sigma$ in measuring $N$ photons  is:

$ \sigma = \sqrt{N}$

The number of expected photons scales linearly with time, i.e., by doubling the exposure time, we expected to recieve twice as many photons. Note here that the noise $\sigma$ \textit{increases} with N, however, the property we are interested in is the signal to noise SN, i.e.:

$SN = \frac{N}{\sigma} = \sqrt{N}$

Therefore, since the expected number of photons N $\propto$ exposure time t, the SN increases with the exposure time as \sqrt{t}.

Additionally, due to the fact that the CCD readout adds noise in each pixel, each pixel has an added noise called the read-out noise (RON). This is generally well known for each instrument.

We will need to combine these errors later on, remember these equations for combining errors:

$A = B + C ,\ \sigma_A = \sqrt{\sigma_B^2 + \sigma_C^2}$

$A = B * C \ or A = \frac{B}{C} ,\  \frac{\sigma_A}{A} = \sqrt{\left(\frac{\sigma_B}{B}\right)^2 + \left(\frac{\sigma_C}{C}\right)^2}$

#### There are two general approaches to phomometry:

$\bullet$ Aperture photometry: is very commonly used in astronomy. It in involves integrating the flux from the sources within a (generally circular) aperture. This is the method we will be using here. We will need to consider the size and placement of the aperture and subtraction of background.

$\bullet$ PSF photometry: this involved fitting a model of the PSF to sources. It can be more accurate than aperture photometry, especially in extremely crowded fields where sources overlap. PSF photometry requires a detailed modelling of the PSF across the field. 

This is the summary of background information needed to complete this tutorial.

<a id="sec:stepbystep"></a>
## 1. Aperture photometry - step by step
Below, you see a simulated image. The colour bar on the right indicates the counts per pixel. Can you locate the source?

In [ ]:
#The line below creates the tutorial image.
myIma = ImageSimulator.tutorial_image()
#You can plot the image like this
myIma.show_ima()

In [ ]:
myIma.get_data()

In [ ]:
myIma._y

In [ ]:
#If you ever need, the data in the image can be accessed like this
test_data = myIma.get_data()
print(test_data)
#This is a numpy array.


You can also plot slices through the image in either x or y as folllows.


In [ ]:
myIma.plot_x(20)

In [ ]:
myIma.plot_y(22)

<div class="alert alert-block alert-success">

### Your turn: Explore the image

Use the slice tool above to explore the image. Where is the source located? What else can you learn using the slice tool?
</div>

<a id="sec:aperture"></a>
## 1.1 Aperture Photometry - Defining the aperture

Having identified the source visually and using the slice tool, we can now define a circular aperture, this is basically a circular mask that defines the area over which we will sum the flux from the source. We will use photutils.CircularAparture as follows:

In [ ]:
#Defining an aperture:
xpos = 23 #guess for x position
ypos = 22 #guess for y position
rad = 4 #aperture radius
aperture1 = photutils.CircularAperture((xpos, ypos), rad)
#Show the initial image again and overplot the aperture
myIma.show_ima()
aperture1.plot(ec='white')

Try redefining the aperture to match the position.

<a id="sec:flux"></a>
## 1.2 Aperture Photometry - Measuring fluxes

Having defined the aperture, we can now perform the aperture photometry, the code below shows how we can calculate the flux in the aperture.

In [ ]:
#The code below shows how we use the function aperture photometry to apply aperture1 to the image myIma._realima
phot = aperture_photometry(myIma.get_data(), aperture1)
#the result is stored in phot, let's have a look
print(phot)


You can access the results as a dictionary, e.g.:

In [ ]:
phot['aperture_sum']

In [ ]:
#We can access the area covered by the aperture as follows
aperture1.area

<div class="alert alert-block alert-success">

### Your turn: Explore the image further. Do you notice anything unusual?
</div>

<div class="alert alert-block alert-success">

### Your turn: Consider how to best choose the size of the aperture

Try varying the aperture radius and replotting. You can also use the slice plotting you learned above to guide your choioce. What do you think is a good choice for aperture?
    
</div>

<a id="sec:sky"></a>
## 1.3 Aperture Photometry - Estimating sky levels

The main contamination to the signal will be sky emission. You might have noticed this already. We can estimate sky levels either by creating an annulus around the object or by estimating the sky in different areas of the image. Below, we show how you can use both using the code provided.

In [ ]:
# defining a sky annulus at position 20, 20 with inner radius 4, outer radius 10
ringsky = photutils.CircularAnnulus((20, 20), 4, 10)
# 
sky_annulus = aperture_photometry(myIma.get_data(), ringsky)
print(sky_annulus)
print('Sky area is :' + str(ringsky.area))
myIma.show_ima()
ringsky.plot(ec='white')

<div class="alert alert-block alert-success">

### Your turn: Consider different sizes for the sky ring. What is the optimal choice?
</div>

## Aperture photometry - different ways to measure sky

Above, we have measured the sky using a ring around the source. We could also measure the sky in a circle. 

In [ ]:
#Defining a sky area 
awaysky = photutils.CircularAperture((30, 27), 4)
#
sky_circle = aperture_photometry(myIma._realima, awaysky)
print(sky_circle)
print('Sky area is :' + str(awaysky.area))
myIma.show_ima()
awaysky.plot(ec='white')

<div class="alert alert-block alert-success">

### Your turn: Try different sky positions and comment on how the sky should be placed.
</div>

<div class="alert alert-block alert-success">

### Your turn: Compare the two methods to measure sky levels.
</div>

<div class="alert alert-block alert-success">

### Your turn: Aperture Photometry - Write a function to calculate the flux of a star given the results of the measurements made.

You have now measured the flux in the aperture and sky aperture. Write a function to caculate the object flux.

<a id="aperturefunction"></a>
### The aperture photometry function
The call below contains the basis to the aperture photometry. Fill in the gaps to write your own function to use for Coursework A.

In [ ]:

def ap_phot(image, x, y, radius, skyx, skyy, skyinner, skyouter):
    #Defining the object aperture
    ap = photutils.CircularAperture((x, y), radius)
    #measuring the flux in the object aperture
    fl_ap = aperture_photometry(image, ap)
    #Defining the sky perture, if syinner=0, create a circular ap, otherwise, create an annulus
    if skyinner == 0:
        ap_sky = photutils.CircularAperture((skyx, skyy), skyouter)
    else:
        ap_sky = photutils.CircularAnnulus((skyx, skyy), skyinner, skyouter)
    #measure the flux in the sky
    fl_sky = aperture_photometry(image, ap_sky)
    #the areas of each of the apertures are
    obj_area = ap.area
    sky_area = ap_sky.area
    
    #Write down the flux in the oject given
    #fl_ap: the flux in the aperture
    #fl_sky: the flux in the sky aperture
    #obj_area: area of object aperture
    #sky_area: area of sky aperture
    
    fl_object = None ##Add your equation here
    
    #Similarly, also calculate the error:
    
    #the shot noise in the main aperture:
    shot_ap = numpy.sqrt(fl_ap)
    #uncertainty in the sky level
    unc_sky = None ## add your equation for the uncertainty in the sky  here
    ron = None ## add your equation for the readout noise here
    
    #add the other sources of error
    err_obj = numpy.sqrt(shot_ap**2 + unc_sky**2 + ron**2)
    
    return fl_object, err_obj

<div class="alert alert-block alert-success">

### Your turn: Aperture Photometry - Write a function to calculate the error 

Using the equations given above, write a function to calculate the error in your flux.
</div>

<a id="sec:practice"></a>
## 2. Aperture photometry: Practice
Below, you can find instructions how to create random images that you can use to test your aperture photometry functions. We will start with a high-SN image with a single star in the field. You can create a random image as follows.

In [ ]:
test_me = ImageSimulator.centred_psf_highSN()

Everytime you rerun the cell above, a different test image is created. You can access the data and show the image as you did before in the tutorial image. This time, you can give a guess for the source position and flux, and the function will then tell you if your estimate is correct. You can plot the x and y slices as was done before.

Here, I am guessing that the source is in the middle of the image, that the source flux is 700, the measurement error in the flux is 50 and the position error is 2.

In [ ]:
test_me.guess_psf(x=25,y=25,flux=700, error=50, poserr=2)

If you are lost, you can get the description of your random image as follows:

In [ ]:
test_me.explain_practiceima()

<a id="sec:practice-noise"></a>
## Aperture Photometry - Advanced practice - noisy Images

If you have gotten a good hold of the practice set above, you can try these advanced examples. The first one gives a noisier version of the one above. The guesses and solutions work as above.

In [ ]:
test_lowSN = ImageSimulator.centred_psf_lowSN()

<a id="sec:practice-crowded"></a>
## Aperture Photometry - Advanced practice - Crowded Fields

The next example is more challenging, this is a good chance to make use of plotting to identify where the sources are.

In [ ]:
test_crowded = ImageSimulator.crowded_field()

If you get stuck, you can show the number of objects in the field as follows:

In [ ]:
test_crowded.show_objectnumber()

You can also reveal the positions.

In [ ]:
test_crowded.show_positions()

<div class="alert alert-block alert-info">

<a id="sec:summary"></a>
## Summary & Coursework Notes

After completing this notebook, you should have:

- applied aperture photometry and error estimation techniques
- written your own photometry function

In <b>Coursework A</b>, you will be assessed on your understanding of these techniques, you will need to apply your photometry funtion to simulated data. You will also need to answer questions on decisions made when choosing aperture photometry paraneters.
</div>